In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [1]:
path_1 = "/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/word2vec_result_final_500.pickle"
path_2 = "/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_result_final_500.pickle"


data = pd.read_pickle((path_1))
# data = data.drop(columns = ['doc_id'])
X_pred = (pd.read_pickle(path_2))

# gc.collect()

NameError: ignored

In [0]:
data.shape

(642290, 502)

In [0]:
X_pred.shape

(9092, 501)

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(449603, 500) (449603,) (192687, 500) (192687,)


In [0]:
print((y_train.value_counts()+y_test.value_counts())/(474108+168182))

0.0    0.738153
1.0    0.261847
Name: importance, dtype: float64


In [0]:
X_pred.drop(columns = ['doc_id'], inplace = True)
X_pred.shape

(9092, 500)

In [0]:
from sklearn.model_selection import RandomizedSearchCV
import time
from scipy.stats import uniform

model = LogisticRegression(max_iter=10000000)

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(model, hyperparameters, random_state=42, n_iter=30, cv=3, verbose=1, n_jobs=-1, scoring = 'f1')

s = time.time()
best_model = clf.fit(X_train, y_train)
e = time.time()

# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print("Model Operation Completed in :",(e-s)/60,"min")

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 23.2min finished


Best Penalty: l2
Best C: 3.795542149013333
Model Operation Completed in : 24.11095559199651 min


In [0]:
model = LogisticRegression(max_iter=10000000, penalty = best_model.best_estimator_.get_params()['penalty'], C = best_model.best_estimator_.get_params()['C'])

import time

s = time.time()
model.fit(X_train, y_train)
e = time.time()

print("Model trained in", (e-s)/60,"min")
print(model.score(X_test, y_test))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

Model trained in 0.9165612697601319 min
0.7608037906034139
0.359629866340623


In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []
acc = []

pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = cutoff, y = f1, mode='lines', name='f1'))
fig.add_trace(go.Scatter(x = cutoff, y = acc, mode='lines', name='accuracy'))
fig.show()

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_sent.pickle")
GS_sent.shape

(9092, 2)

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_sent.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = model.predict(X_pred)
probability_of_predictions = model.predict_proba(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1][:,1]
result['doc_id'] = result['doc_id']

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=0)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
26,admittedly compensation calculated date vestin...,0.0,0.608107
27,court order dated civil revisions conferred ne...,0.0,0.641982
40,agricultural year immediately preceding abolit...,0.0,0.560093
41,therefore deduction income tax gross assets il...,0.0,0.546003
60,bars challenge amount compensation acquisition...,0.0,0.550931
...,...,...,...
9059,basappa v. t. nagappa air sc indlaw sc rupa as...,49.0,0.704057
9060,cannot said hypertechnical manner order passed...,49.0,0.581258
9062,constitution s. act cannot given restricted co...,49.0,0.508752
9069,quintessentiality cannot put state jacket form...,49.0,0.559367
